In [1]:
# Load the dataset.

import datasets # type: ignore

dataset = datasets.load_dataset('jondurbin/truthy-dpo-v0.1', split='train')
# dataset = datasets.load_dataset('ZSvedic/phi3-arena-short-dpo', split='train')
# eval_dataset = datasets.load_dataset('ZSvedic/phi3-arena-short-dpo', split='test')
print(f"Train dataset: {len(dataset)}") #, Test dataset: {len(eval_dataset)}")

Generating train split:   0%|          | 0/1016 [00:00<?, ? examples/s]

Train dataset: 1016


In [2]:
# Load the model and corresponding tokenizer.
import torch # type: ignore

import sys
sys.path.append('..')  # Add the parent directory to the Python path
import utils.llm_utils as llm

# model_name = 'microsoft/Phi-3-mini-4k-instruct'
model_name = 'microsoft/phi-2'
tokenizer, model = llm.load_tokenizer_and_model(model_name)

# I don't think this is needed, but everbody sets ref_model.
from transformers import AutoModelForCausalLM 

ref_model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        torch_dtype=torch.float16, 
        device_map="auto", 
        trust_remote_code=True,
        # Required for packing: https://huggingface.co/blog/packing-with-FA2
        attn_implementation="flash_attention_2")

print(f'Allocated GPU memory: {torch.cuda.memory_allocated() / (1024*1024):,.1f} MB')

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Allocated GPU memory: 10,619.7 MB


In [3]:
model.train()
ref_model.eval()

# Configure DPOTrainer.

from trl import DPOTrainer, DPOConfig

# Set up the training arguments
training_args = DPOConfig(
    output_dir="../results",
    logging_dir="../logs",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    max_prompt_length = 108, # 60 words x 6 ch x 1.2 buffer / 4 chars_in_token
    max_length = 324, # (60+120) words x 6 ch x 1.2 buffer / 4 chars_in_token
    remove_unused_columns=False,
    learning_rate=1e-6,
    # gradient_accumulation_steps=8,
    # gradient_checkpointing=True,
    logging_steps=5,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = DPOTrainer(
    model=model,
    ref_model=ref_model, # Not sure if this is needed?
    beta=0.1,
    tokenizer=tokenizer,
    train_dataset=dataset,
    # eval_dataset=eval_dataset,
    args=training_args)

# For debugging purposes, save the initial model before training.
trainer.save_model("../results/base-model")

Tokenizing train dataset:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [4]:
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
5,0.415900
10,0.000000
15,0.000000


KeyboardInterrupt: 

In [5]:
# Save.
trainer.save_model(training_args.output_dir)